## Classifying digits with Keras

In this notebook, a multi-layer perceptron (MLP) and Convolutional Neural Network (CNN) will be used to classify digits using the Keras package. 

In [1]:
# Imports

from __future__ import print_function
import keras

#from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten
from keras.optimizers import RMSprop

Using TensorFlow backend.


In [5]:
# Set some parameters

batch_size = 128
num_classes = 10 
epochs = 1000

In [6]:
# Instead of the whole MNIST, use sklearn's simplified version
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

digits = load_digits()
n_samples = len(digits['images'])

# Data and targets
data = digits['images'].reshape(n_samples, 64)
targets = digits['target']

# Train/Test
X, X_test, y, y_test = train_test_split(data, targets, test_size=0.33, random_state=111)

# Train/Valid
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.40, random_state=123)

### MLP model

In [7]:
# Normalize digits
X_train /= 255.
X_test /= 255.
X_valid /= 255.

print (X_train.shape, X_valid.shape, X_test.shape)

(721, 64) (482, 64) (594, 64)


In [8]:
# Convert targets into binary class matrices
Y_train = keras.utils.np_utils.to_categorical(y_train, num_classes)
Y_valid = keras.utils.np_utils.to_categorical(y_valid, num_classes)
Y_test = keras.utils.np_utils.to_categorical(y_test, num_classes)

In [9]:
# Construct model
model = Sequential()
model.add(Dense(48, activation = 'relu' ,
                input_shape = (64, )))     # Fully connected 48 neurons
model.add(Dropout(0.5))                    # Dropout layer
model.add(Dense(32, activation = 'relu'))  # Fully connected 32 neurons
model.add(Dropout(0.5))                    # Dropout layer 
model.add(Dense(num_classes, activation = 'softmax')) # Classifier

# Compile
model.compile(loss = 'categorical_crossentropy',
             optimizer = RMSprop(),
             metrics = ['accuracy'])

# Fit
history = model.fit(X_train, Y_train, 
                   batch_size = batch_size, epochs = epochs,
                   verbose = 1, validation_data = (X_valid, Y_valid))


Train on 721 samples, validate on 482 samples
Epoch 1/1000
721/721 [==============================] - 0s - loss: 2.3005 - acc: 0.1331 - val_loss: 2.2949 - val_acc: 0.2822
Epoch 2/1000
721/721 [==============================] - 0s - loss: 2.2941 - acc: 0.1540 - val_loss: 2.2895 - val_acc: 0.2925
Epoch 3/1000
721/721 [==============================] - 0s - loss: 2.2888 - acc: 0.1775 - val_loss: 2.2848 - val_acc: 0.3237
Epoch 4/1000
721/721 [==============================] - 0s - loss: 2.2886 - acc: 0.1526 - val_loss: 2.2799 - val_acc: 0.2178
Epoch 5/1000
721/721 [==============================] - 0s - loss: 2.2792 - acc: 0.1997 - val_loss: 2.2744 - val_acc: 0.2116
Epoch 6/1000
721/721 [==============================] - 0s - loss: 2.2712 - acc: 0.2053 - val_

In [10]:
# Evaluate on the test set
score = model.evaluate(X_test, Y_test, verbose = 0)

print ('Test loss = {:.4f}, Test accuracy = {:.4f}'.format(score[0], score[1]))

Test loss = 0.2014, Test accuracy = 0.9461


### CNN model

In [24]:
# Load the digits data again, do not reshape
digits = load_digits()
n_samples = len(digits['images'])

# Data and targets
data = digits['images'].reshape(-1,8,8,1) # Reshape to have 1 color channel
targets = digits['target']

# Train/Test
X, X_test, y, y_test = train_test_split(data, targets, test_size=0.33, random_state=111)

# Train/Valid
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.40, random_state=123)

In [25]:
# Normalize digits
X_train /= 255.
X_test /= 255.
X_valid /= 255.

# Convert targets into binary class matrices
Y_train = keras.utils.np_utils.to_categorical(y_train, num_classes)
Y_valid = keras.utils.np_utils.to_categorical(y_valid, num_classes)
Y_test = keras.utils.np_utils.to_categorical(y_test, num_classes)

print (X_train.shape, X_valid.shape, X_test.shape)

(721, 8, 8, 1) (482, 8, 8, 1) (594, 8, 8, 1)


In [26]:
# Build CNN model
input_shape = (X_train.shape[1], X_train.shape[2], 1) # One color channel

model = Sequential()
model.add(Conv2D(8, kernel_size = (3,3),
                activation = 'relu',
                padding = 'same',
                input_shape =input_shape )) # First conv layer
model.add(Conv2D(16, kernel_size = (3,3), 
                activation = 'relu',
                padding = 'same'))          # Second conv layer
model.add(MaxPooling2D(pool_size=(2,2)))    # Maxpooling layer
model.add(Dropout(0.25))                    # Dropout layer
model.add(Flatten())                        # Flatten 
model.add(Dense(32, activation = 'relu'))   # Fully Connected layer: 32 neurons
model.add(Dropout(0.5))                     # Dropout
model.add(Dense(num_classes, activation = 'softmax')) # Classifier

# Compile
model.compile(loss = 'categorical_crossentropy', 
             optimizer = RMSprop(),
             metrics = ['accuracy'])

In [30]:
# Fit the model
history = model.fit(X_train, Y_train, 
                    batch_size=batch_size,
                   epochs = 500,
                   verbose = 1, validation_data = (X_valid, Y_valid))

Train on 721 samples, validate on 482 samples
Epoch 1/500
721/721 [==============================] - 0s - loss: 0.0978 - acc: 0.9626 - val_loss: 0.0630 - val_acc: 0.9855
Epoch 2/500
721/721 [==============================] - 0s - loss: 0.0861 - acc: 0.9695 - val_loss: 0.0663 - val_acc: 0.9813
Epoch 3/500
721/721 [==============================] - 0s - loss: 0.0860 - acc: 0.9750 - val_loss: 0.0696 - val_acc: 0.9813
Epoch 4/500
721/721 [==============================] - 0s - loss: 0.1073 - acc: 0.9598 - val_loss: 0.0717 - val_acc: 0.9793
Epoch 5/500
721/721 [==============================] - 0s - loss: 0.0971 - acc: 0.9612 - val_loss: 0.0709 - val_acc: 0.9751
Epoch 6/500
721/721 

In [31]:
# Evaluate on the test set
score = model.evaluate(X_test, Y_test, verbose = 0)

print ('Test loss = {:.4f}, Test accuracy = {:.4f}'.format(score[0], score[1]))

Test loss = 0.0958, Test accuracy = 0.9798
